In [1]:
# SKIP THIS IF YOU WANT TO USE THE NON WEB INTERFACE (can only be done when notebook run locally)
# %matplotlib inline

In [2]:
#%%
try:
    %load_ext autoreload
    %autoreload 2
    print 1
except:
    print 'NOT IPYTHON'

# import matplotlib as mpl
# mpl.use('TKAgg')
from matplotlib import pyplot as plt
# plt.ion()

import sys
import numpy as np
import ca_source_extraction as cse
from scipy.sparse import coo_matrix
import tifffile
from time import time
import psutil

1


In [3]:
import bokeh.plotting as bpl
from bokeh.io import vform,hplot,vplot,gridplot
from bokeh.models import CustomJS, ColumnDataSource, Slider
from IPython.display import display, clear_output
# import matplotlib.cm as cm

bpl.output_notebook()

Loading BokehJS ...

In [4]:
#%%
n_processes = np.maximum(np.int(psutil.cpu_count()*.75),1) # roughly number of cores on your machine minus 1
print 'using ' + str(n_processes) + ' processes'
print "Stopping  cluster to avoid unnencessary use of memory...."
sys.stdout.flush()  
cse.utilities.stop_server()

using 3 processes
Stopping  cluster to avoid unnencessary use of memory....
Stopping cluster...
Waiting for cluster to stop....... done


In [5]:
try:
    t = tifffile.TiffFile('movies/demoMovie3D.tif') 
except:
    # create 3d tiff
    from scipy.io import loadmat
    mat = loadmat('../ROI_detect/Datasets/data_exp3D')
    data = np.transpose(mat['data'], [3, 0, 1, 2])
    data -= data.min()
    data *= (2**15-1)/data.max()
    data = data.astype(np.uint16)
    t = tifffile.TiffWriter('movies/demoMovie3D.tif', imagej=True)
    t.save(np.transpose(data, [1,0, 2, 3]))
    t.close()
    # change properties in ImageJ to 100 frames 1 channels and save changes

In [6]:
Yr = t.asarray().astype(dtype=np.float32) 
T, dims = Yr.shape[0], Yr.shape[1:]
Yr = np.transpose(Yr, range(1,Yr.ndim)+[0])
Yr=np.reshape(Yr,(np.prod(dims),T),order='F')

In [7]:
np.save('Yr',Yr)
Yr=np.load('Yr.npy',mmap_mode='r')  
# Yr,d1,d2,T=cse.utilities.load_memmap(fname_new)
Y=np.reshape(Yr,dims+(T,),order='F')
Cn = cse.utilities.local_correlations(Y)
plt.imshow(Cn.max(0) if len(Cn.shape)==3 else Cn, cmap='gray',vmin=np.percentile(Cn, 1), vmax=np.percentile(Cn, 99))    
plt.show()

In [8]:
#%%
K=30 # number of neurons expected per patch
gSig=[2,2,2] # expected half size of neurons
merge_thresh=0.8 # merging threshold, max correlation allowed
p=2 #order of the autoregressive system
options = cse.utilities.CNMFSetParms(Y,n_processes,p=p,gSig=gSig,K=K)
cse.utilities.start_server(options['spatial_params']['n_processes'])

using 3 processes
Starting cluster.................

In [9]:
options['spatial_params']['method'] = 'dilate'

In [10]:
#%% PREPROCESS DATA AND INITIALIZE COMPONENTS
t1 = time()
Yr,sn,g,psx = cse.pre_processing.preprocess_data(Yr,**options['preprocess_params'])
Atmp, Ctmp, b_in, f_in, center=cse.initialization.initialize_components(Y, **options['init_params'])                                                    
print time() - t1

Running on 3 engines.
Running fft for remaining pixels:2
Noise Normalization
Roi Extraction...
Refining Components...
6.05524086952


In [11]:
refine_components=False
if refine_components:
    Ain,Cin = cse.utilities.manually_refine_components(Y,options['init_params']['gSig'],coo_matrix(Atmp),Ctmp,Cn,thr=0.9)
else:
    Ain,Cin = Atmp, Ctmp

In [12]:
plt.close('all')

In [13]:
p1=cse.nb_plot_contour(Cn.max(0),Ain.reshape(dims+(K,),order='F').max(0).reshape((-1,K),order='F'),
                       dims[1],dims[2],thr=0.9,face_color=None, line_color='black',alpha=0.4,line_width=2)
bpl.show(p1)

In [14]:
#%% UPDATE SPATIAL COMPONENTS
t1 = time()
A,b,Cin = cse.spatial.update_spatial_components(Yr, Cin, f_in, Ain, sn=sn, **options['spatial_params'])
t_elSPATIAL = time() - t1
print t_elSPATIAL 
#clear_output(wait=True)
print('DONE!')

 find search location
Updated Spatial Components
threshold
Computing A_bas
--- 15.8551409245 seconds ---
Remove temporary file created
15.9122409821
DONE!


In [15]:
plt.figure(num=None, figsize=(9, 7), dpi=100, facecolor='w', edgecolor='k')
crd = cse.utilities.plot_contours(
    A.toarray().reshape(dims+(K,),order='F').max(0).reshape((-1,K),order='F'),
                       Cn.max(0),thr=0.9)
plt.show()

In [16]:
plt.close('all')

In [17]:
p1=cse.nb_plot_contour(Cn.max(0),Ain.reshape(dims+(K,),order='F').max(0).reshape((-1,K),order='F'),
                       dims[1],dims[2],thr=0.9,face_color=None, line_color='black',alpha=0.4,line_width=2)
bpl.show(p1)

In [18]:
plt.close()
t1 = time()
options['temporal_params']['p'] = 0 # set this to zero for fast updating without deconvolution
C,f,S,bl,c1,neurons_sn,g,YrA = cse.temporal.update_temporal_components(Yr,A,b,Cin,f_in,bl=None,c1=None,sn=None,g=None,**options['temporal_params'])
t_elTEMPORAL = time() - t1
print t_elTEMPORAL  
clear_output(wait=True)

3 out of total 30 temporal components updated
5 out of total 30 temporal components updated
13 out of total 30 temporal components updated
30 out of total 30 temporal components updated
3 out of total 30 temporal components updated
5 out of total 30 temporal components updated
14 out of total 30 temporal components updated
30 out of total 30 temporal components updated
stopping: overall temporal component not changing significantly
0.330686807632


In [19]:
#%% merge components corresponding to the same neuron
t1 = time()
A_m,C_m,nr_m,merged_ROIs,S_m,bl_m,c1_m,sn_m,g_m=cse.merging.merge_components(Yr,A,b,C,f,S,sn,options['temporal_params'], options['spatial_params'], bl=bl, c1=c1, sn=neurons_sn, g=g, thr=0.8, mx=50, fast_merge = True)
t_elMERGE = time() - t1
print t_elMERGE  

********** No neurons merged! ***************
0.0046808719635


In [20]:
#refine spatial and temporal components
t1 = time()
A2,b2,C2 = cse.spatial.update_spatial_components(Yr, C_m, f, A_m, sn=sn, **options['spatial_params'])
options['temporal_params']['p'] = p # set it back to original value to perform full deconvolution
C2,f2,S2,bl2,c12,neurons_sn2,g21,YrA = cse.temporal.update_temporal_components(Yr,A2,b2,C2,f,bl=None,c1=None,sn=None,g=None,**options['temporal_params'])
print time() - t1
clear_output(wait=True)
print time() - t1 # 100 seconds
print('DONE!')

31.1393790245
DONE!


In [21]:
plt.figure(num=None, figsize=(9, 7), dpi=100, facecolor='w', edgecolor='k')
A_or, C_or, srt = cse.utilities.order_components(A2,C2)
crd = cse.utilities.plot_contours(coo_matrix(A_or.reshape(dims+(K,),order='F').max(0).reshape((-1, K),order='F')),
                                  Cn.max(0),thr=0.9)
plt.show()

In [22]:
p2=cse.utilities.nb_plot_contour(Cn.max(0),A_or.reshape(dims+(K,),order='F').max(0).reshape((-1, K),order='F'),dims[1],dims[2],thr=0.9,face_color='purple', line_color='black',alpha=0.3,line_width=2)
bpl.show(p2)

In [23]:
traces_fluo=cse.utilities.nb_view_patches(Yr.reshape(dims+(T,),order='F').max(0).reshape((-1,T),order='F'),
                                          A_or.reshape(dims+(K,),order='F').max(0).reshape((-1, K),order='F'),
                                          C_or,b2.reshape(dims+(1,),order='F').max(0).reshape((-1, 1),order='F'),
                                          f2,dims[1],dims[2],thr = 0.9,image_neurons=None)

Loading BokehJS ...

In [24]:
#%% STOP CLUSTER
cse.utilities.stop_server()

Stopping cluster...
Waiting for cluster to stop....... done
